## OVERVIEW
This project demonstrates how to interact with NASA's public APIs to fetch and analyze data. Specifically, it covers two main use cases:

Fetching and displaying the Astronomy Picture of the Day (APOD).
Retrieving data on near-Earth objects (asteroids) using the NeoWs (Near-Earth Object Web Service) and processing it for analysis.
By working on this project, you will learn how to consume data from public APIs, handle JSON responses, process and clean the data, and save your results to a file. The project is implemented in Python using popular libraries such as Requests, Pandas, and Matplotlib.

###To get your **NASA API key**, follow these steps:


1. **Visit the NASA API Portal:**
   - Go to the [NASA Open APIs website](https://api.nasa.gov/).

2. **Sign Up for an API Key:**
   - Scroll down to the section titled **"Get Started"**.
   - Click on **"Get Your API Key"**.

3. **Fill Out the Registration Form:**
   - You will be directed to a form where you need to provide your **name**, **email address**, and other basic information.
   - Complete the form and click **"Submit"**.

4. **Receive Your API Key:**
   - After submitting the form, you will receive an **email** from NASA containing your **unique API key**.
   - You can also see the key on the confirmation page after submitting the form.

5. **Save the API Key:**
   - Keep this key safe. You will use it as a **user identifier** when making requests to NASA's public APIs.

### **Using the API Key:**
Once you have your API key, you can add it to your code:

```python
api_key = "YOUR_API_KEY_HERE"
```

### **Notes:**
- **Usage Limits:** NASA's free API key has a rate limit of **1,000 requests per hour**. If you need higher limits, you can reach out to NASA to discuss your requirements.
- **Keep Your Key Secure:** Avoid sharing your API key publicly. If you're sharing code or projects, consider storing the API key in environment variables or a configuration file that's not shared publicly.



In [ ]:
import requests
from IPython.display import Image, display
import pandas as pd
from PIL import Image as PILImage
import io
import matplotlib.pyplot as plt
import os

# Define Constants
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual API key
APOD_URL = "https://api.nasa.gov/planetary/apod"
NEO_WS_URL = "https://api.nasa.gov/neo/rest/v1/neo/browse"

# Function to fetch Astronomy Picture of the Day (APOD)
def fetch_apod_image(api_key):
    params = {
        "api_key": api_key,
        "hd": True  # Request high-resolution images
    }

    try:
        response = requests.get(APOD_URL, params=params)
        if response.status_code == 200:
            data = response.json()

            if data['media_type'] == 'image':
                # Display the image using URL
                image_url = data['url']
                display(Image(url=image_url))
                print("Astronomy Picture of the Day displayed successfully.")
            else:
                print("Today's APOD is not an image. It might be a video or other media.")
        else:
            print(f"Failed to retrieve APOD data. Error: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Function to display APOD using Matplotlib for more control
def display_apod_with_matplotlib(api_key):
    params = {
        "api_key": api_key,
        "hd": True
    }

    try:
        response = requests.get(APOD_URL, params=params)
        if response.status_code == 200:
            data = response.json()

            if data['media_type'] == 'image':
                image_url = data['url']
                image_response = requests.get(image_url)
                image_data = PILImage.open(io.BytesIO(image_response.content))

                plt.imshow(image_data)
                plt.axis('off')
                plt.show()
                print("APOD Image displayed using Matplotlib.")
            else:
                print("Today's APOD is not an image. It might be a video or other media.")
        else:
            print(f"Failed to retrieve APOD data. Error: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Function to fetch asteroid data from NASA NeoWs API
def fetch_asteroid_data(api_key):
    params = {
        "api_key": api_key
    }

    try:
        response = requests.get(NEO_WS_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            asteroids_data = []

            # Extract necessary information
            for asteroid in data.get('near_earth_objects', []):
                asteroid_id = asteroid.get('id')
                asteroid_name = asteroid.get('name')
                min_diameter_km = asteroid.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min')
                abs_magnitude = asteroid.get('absolute_magnitude_h')
                relative_velocity = asteroid.get('close_approach_data', [{}])[0].get('relative_velocity', {}).get('kilometers_per_second')

                asteroids_data.append({
                    'Asteroid ID': asteroid_id,
                    'Asteroid name': asteroid_name,
                    'Minimal estimated diameter (km)': min_diameter_km,
                    'Absolute magnitude': abs_magnitude,
                    'Relative velocity (km/s)': relative_velocity
                })

            # Create DataFrame
            asteroids_df = pd.DataFrame(asteroids_data)
            print("Asteroid Data DataFrame successfully created.")
            return asteroids_df
        else:
            print(f"Failed to retrieve asteroid data. Error: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request failed: {e}")

# Function to save DataFrame to CSV
def save_dataframe_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False)
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Failed to save data to CSV. Error: {e}")

# Main Execution
if __name__ == "__main__":
    # Fetch and display APOD image
    fetch_apod_image(API_KEY)

    # Optionally, display APOD using Matplotlib
    display_apod_with_matplotlib(API_KEY)

    # Fetch and process asteroid data
    asteroid_data_df = fetch_asteroid_data(API_KEY)

    if asteroid_data_df is not None:
        print(asteroid_data_df.head(10))  # Display top 10 entries

        # Save to CSV
        save_dataframe_to_csv(asteroid_data_df, "asteroid_data.csv")
